In [ ]:
import os,sys,datetime
import matplotlib.pyplot as plt
sys.path.append(os.getcwd())

#Module imports
import geeViz.getImagesLib as getImagesLib
import geeViz.taskManagerLib as taskManagerLib
import geeViz.changeDetectionLib as changeDetectionLib
ee = getImagesLib.ee
Map = getImagesLib.Map
Map.clearMap()
##################################################

mtbs = ee.ImageCollection("USFS/GTAC/MTBS/annual_burn_severity_mosaics/v1")
##################################################

tcc = ee.ImageCollection('projects/nlcd-tcc/assets/CONUS-TCC/Final-products/tcc_rawAnalytic')
tcc = tcc.filter(ee.Filter.stringContains('system:index','_mean'))
tcc_years = tcc.aggregate_histogram('Year').keys().map(lambda n:ee.Number.parse(n).int16()).getInfo()
tcc = ee.ImageCollection([tcc.filter(ee.Filter.calendarRange(yr,yr,'year')).mosaic().set('system:time_start',ee.Date.fromYMD(yr,6,1).millis()) for yr in tcc_years])
##################################################
# Min number of years from all available TCC years to start and stop with the MTBS collection
pre_min = 2
post_min = 5

mtbs_years = tcc_years[pre_min:-post_min+1]
print('All TCC Years:',tcc_years)
print('Included MTBS Years:',mtbs_years)

# Iterate through each mtbs year and then tcc year
# Mask the tcc year by the mtbs burn mask for the given mtbs year and set relevant properties
zCollection = []
for mtbs_year in mtbs_years:
	mtbs_yr = mtbs.filter(ee.Filter.calendarRange(mtbs_year,mtbs_year,'year')).mosaic()
	for tcc_year in tcc_years:
		zYr = tcc_year-mtbs_year
		tcc_yr = tcc.filter(ee.Filter.calendarRange(tcc_year,tcc_year,'year')).mosaic().updateMask(mtbs_yr)
		tcc_yr = tcc_yr.set({'tcc_year':tcc_year,'mtbs_year':mtbs_year,'zYr':zYr})
		zCollection.append(ee.Image(tcc_yr))

zCollection = ee.ImageCollection(zCollection)

# Find the z years available and mosaic based on those years
zYrs = zCollection.aggregate_histogram('zYr').keys().map(lambda n:ee.Number.parse(n).int16()).sort()
def getZYr(zYr):
	zYr = ee.Number(zYr)
	zCYr = zCollection.filter(ee.Filter.eq('zYr',zYr)).mosaic()
	d = ee.Date.fromYMD(zYr.add(1900),6,1).millis()
	return zCYr.set({'zYr':zYr,'system:time_start':d})
zCollection = ee.ImageCollection(zYrs.map(getZYr))

Map.addTimeLapse(zCollection,{'min':0,'max':80,'palette':'EEE,080'},'TCC Z MTBS')

summaryArea = ee.Geometry.Polygon(
          [[[-116.35310363752222, 44.63158113370513],
          [-116.35310363752222, 43.351201397586905],
          [-113.62849426252222, 43.351201397586905],
          [-113.62849426252222, 44.63158113370513]]], None, False)
		
zBns = zYrs.map(lambda bn:ee.Number(bn).format())
print(zBns.getInfo())
tccStack = zCollection.toBands().rename(zBns)
print('The stack band names are: {}'.format(tccStack.bandNames().getInfo()))

# Get zonal mean
tccMeans = tccStack.reduceRegion(ee.Reducer.mean(), summaryArea, 30, 'EPSG:5070', None, True, 1e13, 4).getInfo() #gee reduceRegion applies a reducer to all pixels
print(tccMeans.keys())
plt.plot(zYrs.getInfo(),tccMeans.values())

Map.turnOnInspector()
# Map.view()